In [3]:
import os
import sys
import csv
import copy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
csv_root = "../paper/benchmark/data/benchmark.csv"
device_list = ["mate30e_cpu","mate30e_gpu","mate30e_npu"]
type = "benchmark"

In [15]:
def get_data(type,device,dataA,dataB):
    # type = "64_DWConv_kernel"
    data_root = "../paper/{}/data".format(type)
    # device_list= ["mi11_cpu","redmi_gpu","mate30e_npu"]
    # device = "mate30e_npu"
    txt = open('{}/{}.txt'.format(data_root,device), 'r')

    latency = txt.readlines()
    latency = [i.strip() for i in latency]

    info, data = latency[:3], latency[3:]
    latency_dict = {}
    for i in data:
        tmp = i.split(" ")
        (filepath, filename) = os.path.split(tmp[0])
        (name, suffix) = os.path.splitext(filename)
        latency_dict[name] = [float(tmp[1]),float(tmp[2]),float(tmp[3])]
    read_csv   = '{}/{}.csv'.format(data_root,type)
    # write_csv = 'dw_{}_{}_{}.csv'.format("kernel",device,"core0")
    write_csv  = '{}/benchmark_{}.csv'.format(data_root,device)
    read_file  = open(read_csv,"r", encoding='utf-8')
    write_file = open(write_csv,"w+", encoding='utf-8')
    reader = csv.reader(read_file)
    writer = csv.writer(write_file)
    title = ["op","H","in_c","out_c","stride","kernel_size","FLOPs","Params","MACs", 'load_time', 'first_time', 'avg_time','intensity','FLOPS','bandwidth']
    writer.writerow(title)
    for row in reader:
        if row[0] == "op":
            continue
        if row[0] == 'Conv':
            name = "Conv_FLOPS"
        elif row[0] =='DWConv':
            name = "DWConv_Bandwidth"
        # if name in latency_dict.keys():
        load_time  = latency_dict[name][0]
        first_time = latency_dict[name][1]
        avg_time   = latency_dict[name][2]
        intensity = float(row[6]) / float(row[8])
        FLOPS = float(row[6]) / (avg_time/1000)
        bandwidth = float(row[8]) / (avg_time/1000)
        tmp = copy.deepcopy(row)
        # tmp.pop(2)
        # tmp.pop(0)
        tmp.append(load_time)
        tmp.append(first_time)
        tmp.append(avg_time)
        tmp.append(intensity)
        tmp.append(FLOPS)
        tmp.append(bandwidth)
        writer.writerow(tmp)
    write_file.close()
    def get_col(data,label):
        x = data[label]
        y = [i for i in x]
        return y


    read_file = open(write_csv,"r", encoding='utf-8')
    data = pd.read_csv(read_file)
    dataA = get_col(data, dataA)
    dataB = get_col(data, dataB)
    
    return dataA,dataB


In [25]:
device = "mate30e_npu"
FLOPS,Bandwidth = get_data("benchmark",device,"FLOPS","bandwidth")
FLOPS = [i/1024 for i in FLOPS]
print(FLOPS,Bandwidth)
print(max(FLOPS),max(Bandwidth))

[244785.937195738, 125.0185741881651] [1639.19154372146, 2500.4113493800714]
244785.937195738 2500.4113493800714
